In [1]:
import pandas as pd
import numpy as np
import math
import pyproj
from datetime import datetime, timedelta

In [2]:
# Change default warnings
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

## This notebook covers:
# 📈 Data Acquisition, Cleaning, Predictor and Outcome Variable Creation 📉
Note this does not include anything realted to SST or Wind Shear, as they are given their own notebook 😬

### 1) Read in data from official HURDAT2.csv file

Updated 10 June 2021 to include the 2020 hurricane season. These methods read in the raw HURDAT2 file, convert it to a lines object, and then to a dataframe. The method `hurdat_lines_to_df` adds the storm identifier to each row, as described in Note 1 at the end of this notebook.

In [3]:
def read_hurdat_lines():
    """
    read in official NHC HURDAT2.csv data file as line oject; 
    args: none; returns: line object
    """
    f = open("../data/HURDAT2.csv", "r")
    lines = f.readlines()
    f.close()
    return lines

In [4]:
def hurdat_lines_to_df(lines):
    """
    convert HURDAT lines object to dataframe, accounting for necessary formating of HURDAT data file, see note (1);
    args: lines object; returns: df with all storm observations, in tidy format(?)
    """
    hurdat = []     # to store all observations as nested list
    storm_info = [] # to store name and storm code
    df = pd.DataFrame()
    for line in lines:
        arr = line.split(",")
        
        # If this is a new storm, it will have "AL" in the first item ('AL' for Atlantic stroms)
        # Since this is a new storm, we need to update storm info and not add this 'observation' to list
        if "AL" in arr[0]: 
            storm_info = [arr[0],arr[1].strip()]
            
        # If this is the same storm as previous row, add new observation to list of others
        else:
            arr.insert(0,storm_info[0])
            arr.insert(1,storm_info[1])
            hurdat.append(arr)
            
    # Create and return dataframe from the nested list of observations
    df = pd.DataFrame(hurdat)
    return df

### 2) Beautify the Dataframe (Rename, Strip, Retype)

Make the dataframe easier to work with by cleaning it up in several ways

In [5]:
def rename_columns(df):
    """ rename columns from HURDAT file, see note (1); returns given dataframe """
    # note we are leaving the storm radii columns (index 10-21) alone for now, we will deal with those later
    col_names = {
        df.columns[0]: 'Storm_Identifier',
        df.columns[1]: 'Storm_Name',  
        df.columns[2]: 'Date',
        df.columns[3]: 'Time', 
        df.columns[4]: 'Record',
        df.columns[5]: 'Status',
        df.columns[6]: 'Lat',  
        df.columns[7]: 'Lon',   
        df.columns[8]: 'Wind',
        df.columns[9]: 'Pressure'
    }
    df = df.rename(columns = col_names) # rename columns according to dictionary
    return df

In [6]:
def strip_string_columns(df):
    """ Strip extra spaces on all object columns from raw HURDAT file; returns given dataframe """
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()) # strip spaces from string columns
    return df

In [7]:
def retype_columns(df):
    """ convert column types into something more useful; returns given dataframe """
    df[['Wind','Pressure']] = df[['Wind','Pressure']].astype(str).astype(int) # convert from string to integer
    return df

#### HURDAT stores Date and Time seperately, but combining them into one datetime column makes life soo much better 😜

In [8]:
def create_datetime_column(df):
    """
    HURDAT data stores date and time in seperate columns, but we combine into one DateTime;
    removes seperate date and time columns; returns given dataframe;
    """
    df["DateTime"] = df["Date"] + ' ' + df["Time"] # combine string columns
    df["DateTime"] = pd.to_datetime(df["DateTime"], format = '%Y%m%d %H%M') # convert to datetime object
    df['YearDay'] = df.apply(lambda row: row.DateTime.timetuple().tm_yday, axis=1) # day of the year
    df = df.drop(columns=['Date', 'Time'], axis=1) # remove unneeded columns
    return df

### 3) Convert coordinates from string (19.7W) to float (-19.7)

R (and others) understand much coordinates better as numbers instead of text, so we convert the string coordinates to numerics 

In [9]:
def convert_coordinates(df):
    """
    convert HURDAT string coordinates to something more useable (float between -180 and 180); 
    string coords have number and direction, so we split and multiply by -1 depending on hemisphere;
    returns given dataframe
    """
    for direc in ['Lat','Lon']: # loop for both coordinate types
        # get the corrdiante's direction string (N,E,S,W) and numeric value
        df[f'{direc}_Hemisphere'] = df[f'{direc}'].str[-1:]
        df[f'{direc}'] = df[f'{direc}'].str[:-1].astype(float)
        
        # function to multiply value depending on direction string
        convert_direc = lambda row: row[f"{direc}"]*-1 if row[f"{direc}_Hemisphere"] in ['S','W'] else row[f"{direc}"]
        
        # apply lambda func to get final numeric coordinate
        df[f'{direc}'] = df.apply(convert_direc, axis=1)
        
        # remove old coordinate column
        df = df.drop(columns=[f"{direc}_Hemisphere"]) 
    return df

### 4) Wind Speed Acceleration

The `Acceleration` column gives the average acceleration of wind speed (in kts/hr) since the last observation. 

For example: If Storm X has a wind speed of 100 kts at 1200 and increased to 120 kts at 1800, it would have an average acceleration of 20/6 = 3.333 kts/hr.

Note: This calculation is made difficult by the sometimes inconsistent interval between observations. Observations usually come every 6 hours, but if the storm makes landfall inbetween, there will be a new observation at that time. In theory, I could remove observations not on the 6-hour but I don't want to do that.

In [10]:
def difference_in_hours(dt1,dt2):
    """
    calculate the difference between two datetimes in hours, copied from stack; 
    args: dt1 (datetime), dt2 (datetime); returns: hours between dt1 and dt2;
    """
    difference = dt1 - dt2 # get difference in datetime format
    days, seconds = difference.days, difference.seconds # extract days and seconds bc hours is not native(??)
    hours = days * 24 + seconds // 3600 # calculate hours
    return hours

In [11]:
def calculate_acceleration(delta_wind, delta_time):
    """
    calculate the acceleration per hour for a single strom interval;
    args: 
        delta_wind (float): the change in wind in an interval
        delta_time (float): the length in hours of the interval
    returns: acceleration per hour (float) of wind during the interval
    """
    try:
        accel = delta_wind/delta_time
    except:
        accel = 0 # if delta_time = 0, let acceleration = 0
        
    # a 25 mph change in one hour is far too high to be plausible, so return 0 in this case bc something went wrong
    accel = accel if abs(accel) < 25 else 0
    
    return accel 

In [12]:
def create_acceleration_column(df):
    """
    storm observations usually come every 6 hours, though not always. this method calculates the change in 
    acceleration since the last observation for that storm;
    returns the given dataframe, with a new column for accleration
    """
    df[f"Acceleration"] = 0 # set default acceleration to 0
    storm_codes = df.Storm_Identifier.unique() # get all unique storms
    
    # iterate through each storm code, create a dataframe for each strom
    for index_storm, storm in enumerate(storm_codes):
        df_storm = df[df.Storm_Identifier == storm] # get strom specific df
        current_wind = previous_wind = 0 # set default values to 0
        first_index = np.inf; # reset index to high number
        
        # loop through each row of the storm df
        for index_row, row in df_storm.iterrows():
            # record index of first row in storm df, to get assigned 0 acceleration later
            if index_row < first_index: first_index = index_row
            try:
                # get current weather values
                current_wind = df.iloc[index_row]['Wind']
                current_time = df.iloc[index_row]['DateTime']
                
                # get weather values for previous row
                previous_wind = df.iloc[index_row-1]['Wind']
                previous_time = df.iloc[index_row-1]['DateTime']
                
                # calculate change in time and wind, used to calculate accleration
                delta_wind = current_wind - previous_wind
                delta_time = difference_in_hours(current_time,previous_time)
                acceleration = calculate_acceleration(delta_wind,delta_time)
            except:
                acceleration = 0 # if any of the above failed, then set acceleration to 0
            df.loc[index_row,'Acceleration'] = acceleration # set acceleration in full dataframe
        df.loc[first_index,'Acceleration'] = 0 # set acceleration of first row in storm df to 0
        first_index = np.inf; # reset index of first row
    return df

### 5) Calculate Radii of High Intensity

The last set of columns (the ones we ignored earlier) described the 'size' of the storm. Specifically, they give the maximum extent (in nautical miles) of a certain wind speed in one quadrant. Because twelve size variables is far many, we average these four quadrants together to get the average maximum extent of the three wind speeds, 34kt, 50kt, and 64 kts. This tells us the average radius of the storm with wind speeds above these marks, giving us three proxies for 'storm size'.

In [13]:
def calculate_wind_radii(df):
    """
    calculate the average radius of different wind speed by average the wind extent in the four quadrants;
    HURDAT data has extent of 34,50,64kt winds in 4 quadrants (NW,SW,SE,NE), we average them to just 3 numbers;
    returns given dataframe, with 3 new columns for extent of 34,50,64kt winds
    """
    # get average of 4 quadrant observations for 3 wind categories, and record as single column
    df["34kt_radius"] = df[[10,11,12,13]].astype(str).astype(int).mean(axis=1) 
    df["50kt_radius"] = df[[14,15,16,17]].astype(str).astype(int).mean(axis=1)
    df["64kt_radius"] = df[[18,19,20,21]].astype(str).astype(int).mean(axis=1)
    df.drop(columns = [10,11,12,13,14,15,16,17,18,19,20,21,22], inplace=True) # drop unneeded columns
    return df

### 6) Calculate Bearing, Distance, and Storm Speed

Create additional columns that may prove useful for the machine learning model, including:
- Bearing: the bearing (direction) the storm is moving in (between -180 and 180)
- Distance: the distance the storm has traveled since the last observation in nautical miles
- Speed: the speed of the storm track (not wind speed) in knots

In [14]:
def create_bearing_distance_speed_columns(df):
    geodesic = pyproj.Geod(ellps='WGS84')
    df[f"Bearing"] = 0 # set default bearing to 0
    df[f"Distance"] = 0 # set default bearing to 0
    df[f"Speed"] = 0 # set default bearing to 0
    
    storm_codes = df.Storm_Identifier.unique() # get all unique storms
    # iterate through each storm code, create a dataframe for each strom
    for index_storm, storm in enumerate(storm_codes):
        df_storm = df[df.Storm_Identifier == storm] # get strom specific df
        current_bearing = previous_bearing = 0 # set default values to 0
        first_index = np.inf; # reset index to high number
        
        # loop through each row of the storm df
        for index_row, row in df_storm.iterrows():
            
            # record index of first row in storm df, to get assigned 0 later
            if index_row < first_index: 
                first_index = index_row
                
            try:
                # calculate change in time since last obersvation
                current_time = df.iloc[index_row]['DateTime']
                previous_time = df.iloc[index_row-1]['DateTime']
                delta_time = difference_in_hours(current_time,previous_time)
                
                # get current position
                current_position = [df.iloc[index_row]['Lat'],df.iloc[index_row]['Lon']]
                
                # get previous position
                previous_position = [df.iloc[index_row-1]['Lat'],df.iloc[index_row-1]['Lon']]
                
                # use geodesic to calculate bearing and distance between observations
                bearing, back_bearing, distance = geodesic.inv(
                    previous_position[1], previous_position[0], current_position[1], current_position[0])
                
                # calculate distance and speed in nautical miles
                distance = distance/1852 # 1 nautical mile = 1852 meters
                speed = distance / delta_time
                
            except:
                # if any error occured, let all values equal 0
                bearing = 0; distance = 0; speed = 0;
                
            # set the values for the current row
            df.loc[index_row,'Bearing'] = bearing
            df.loc[index_row,'Distance'] = distance
            df.loc[index_row,'Speed'] = speed
            
        # set the values for the first row of storm
        df.loc[first_index,'Bearing'] = 0
        df.loc[first_index,'Distance'] = 0
        df.loc[first_index,'Speed'] = 0
        
        # reset index of first row
        first_index = np.inf; 
    return df

### 7) Add current storm category
Create a variable denoting the storms current category based on the Saffir-Simpson Hurricane Wind Scale. If not a hurricane, assign Category 0.

In [15]:
def calculate_category(wind):
    """
    args: wind speed; returns: storm category (0 to 5) based on the given wind speed
    """
    cat_breaks = [64,83,96,113,137,300]
    for i,cat in enumerate(cat_breaks):
        if wind > 0 and wind < cat:
            return i
    return 0

In [16]:
def create_category_column(df):
    """
    add column to dataframe denoting the current hurricane category of the storm; returns given dataframe;
    """
    df['Category'] = df.apply(lambda row: calculate_category(row.Wind), axis=1)
    return df

### 8) Remove storms that do not reach Category X

Not all storms in HURDAT dataset are hurricanes, there are also distrubrances, waves, tropical storms and more. In fact, more than half of the storms do not reach the 64kt barrier to be classified as a Category 1 hurricane. Beyond that, only 1/3 of all hurricanes are considered 'major', making it to category 3, 4 or 5.

This method removes storms from the dataframe if they do not reach the given category.

In [17]:
def remove_storms_below_cat_X(df, X):
    """
    remove storms from the full dataset if the storm did not reach category X;
    args: df: full strom df, X (int): remove stroms if they do not meet category X
    """
    if X not in range(1,6): return df # if not a valid storm category, retrun the full df
    
    # minimum wind speed for each category hurricane
    cat_X_minimum_wind_speeds = {1:64 , 2:83, 3:96, 4:113, 5:137} 
    
    # compare max wind speed of strom with desired category miniumum, remove storm if it does not reach threshold
    df = df[(df.groupby('Storm_Identifier')['Wind'].transform('max')) >= cat_X_minimum_wind_speeds[X]]
    return df

### 9) Create `Rapid_NHC24` column

This column will denote whether a storm was Rapid Increasing (per NHC, defined as an increase of 30 knots in maximum sustained wind speed over the previous 24 hours) at the time of the observation.

In [18]:
def within_k_hours_before(time, series, k):
    """
    get items in series whose datetime is within k before hours of argument time;
    args: time (datetime), series (series) of HURDAT observations;
    return: series with items whose datetime is within k hours before of the argument time
    """
    return (time-timedelta(hours=k) <= series) & (series <= time)  

In [19]:
def within_k_hours_after(time, series, k):
    """
    get items in series whose datetime is within k after hours of argument time;
    args: time (datetime), series (series) of HURDAT observations;
    return: series with items whose datetime is within k hours after of the argument time
    """
    return (time <= series) & (series <= time+timedelta(hours=k))

In [20]:
def create_rapid_NHC24_column(df):
    """
    create boolean column based on if storm was 'Rapidly Increasing' per NHC defn. at the time of the observation;
    returns given dataframe of strom observations;
    """
    storm_codes = df.Storm_Identifier.unique() # list of unique stroms
    for index_storm, storm in enumerate(storm_codes): # iterate through all stroms 
        df_storm = df[df.Storm_Identifier == storm] # get df for each storm
        for index_row, row in df_storm.iterrows():
            # get current time and wind for the row
            current_time = row['DateTime']; 
            current_wind = row["Wind"] 

            # find all observations within 24 hours of current observation
            df_k_hours_before = df_storm[ within_k_hours_before(current_time, df_storm['DateTime'], 24) ]

            # get minimum wind within k hours
            min_wind_k_hours = df_k_hours_before["Wind"].min() 

            # determine if strom was RI if it increased by 30mph in the last k hours
            df.loc[index_row,f"Rapid_NHC24"] = (current_wind - min_wind_k_hours ) > 30 
          
    return df

### 10) Create Outcome Variables for Machine Learning Model

Here we create several different outcome variable to used in different machine learning models later in the analysis.
1. `Outcome_Total`: this variable denotes whether the storm will rapidly increase in the next 24 hours
2. `Outcome_Switch`: this variable denotes whether the storm will switch from non-rapidly increasing to rapidly increasing in the next 24 hours
3. `Outcome_HoursToRapid`: this variable denoted the number of hours until the storm will be rapidly increasing. If the storm does not become rapidly increasing after that observation, the varible is assigned NaN.

In [21]:
def create_outcome_columns(df):
    """
    creates the outcome variables used in the machine learning models later on; 
    three outcome variables are created and described in the text above;
    returns given dataframe with three new columns: Outcome_Total, Outcome_Switch, and Outcome_HoursToRapid
    """
    storm_codes = df.Storm_Identifier.unique() # list of unique stroms
    for storm in storm_codes: # iterate through all stroms 
        df_storm = df[df.Storm_Identifier == storm] # get df for each storm
        
        for index_row, row in df_storm.iterrows():
            # get current time and RI status of observation
            current_time = row['DateTime'];
            current_status = row[f"Rapid_NHC24"]
            
            # create dataframe with all observatinos within 24 hours after the current one
            df_24_hours_after = (df_storm[ within_k_hours_after(current_time, df_storm['DateTime'], 24) ])
            
            # calculate outcome variables, see above
            df.loc[index_row, "Outcome_Total"] = True in df_24_hours_after[f'Rapid_NHC24'].tolist()
            df.loc[index_row, "Outcome_Switch"] = (not current_status and True in df_24_hours_after[f'Rapid_NHC24'].tolist())
            
            # not efficient but someone the only effective method
            for k in range(600,-1,-6):
                # get dataframe with all observations with k hours of the current one
                df_k_hours_after = (df_storm[ within_k_hours_after(current_time, df_storm['DateTime'], k) ])
                
                # if any observations in the dataframe are rapidly intensifying, change outcome column to k
                if (True in df_k_hours_after[f'Rapid_NHC24'].tolist()):
                    df.loc[index_row, "Outcome_HoursToRapid"] = k  
    return df

# Main

In [22]:
# Read in data
lines = read_hurdat_lines()
df = hurdat_lines_to_df(lines)

In [23]:
# Clean up dataframe
df = rename_columns(df)
df = strip_string_columns(df)
df = create_datetime_column(df)
df = retype_columns(df)

# Create new columns from calculations
df = create_acceleration_column(df)
df = convert_coordinates(df)
df = create_bearing_distance_speed_columns(df)
df = create_category_column(df)
df = calculate_wind_radii(df)
df = create_rapid_NHC24_column(df)
df = create_outcome_columns(df)

# Remove unneeded stroms
# df = remove_storms_below_cat_X(df,1)
# df = df.reset_index(drop=True)

# Create Pickle
d = datetime.today()
df.to_pickle(f"{d.month}_{d.day}_{d.year}.pkl")

In [24]:
df = pd.read_pickle("12_15_2021.pkl")
df

Storm_Identifier Storm_Name Record Status   Lat   Lon  Wind  Pressure  \
0             AL011851    UNNAMED            HU  28.0 -94.8    80      -999   
1             AL011851    UNNAMED            HU  28.0 -95.4    80      -999   
2             AL011851    UNNAMED            HU  28.0 -96.0    80      -999   
3             AL011851    UNNAMED            HU  28.1 -96.5    80      -999   
4             AL011851    UNNAMED      L     HU  28.2 -96.8    80      -999   
...                ...        ...    ...    ...   ...   ...   ...       ...   
52712         AL312020       IOTA            HU  13.7 -84.7    75       965   
52713         AL312020       IOTA            TS  13.7 -85.7    55       988   
52714         AL312020       IOTA            TS  13.8 -86.7    40      1000   
52715         AL312020       IOTA            TS  13.8 -87.8    35      1005   
52716         AL312020       IOTA            TD  13.7 -89.0    25      1006   

                 DateTime  YearDay  ...   Distance      Speed  Category  \
0     1851-06-25 00:00:00      176  ...   0.000000   0.000000         1   
1     1851-06-25 06:00:00      176  ...  31.866664   5.311111         1   
2     1851-06-25 12:00:00      176  ...  31.866664   5.311111         1   
3     1851-06-25 18:00:00      176  ...  27.209413   4.534902         1   
4     1851-06-25 21:00:00      176  ...  16.999238   5.666413         1   
...                   ...      ...  ...        ...        ...       ...   
52712 2020-11-17 12:00:00      322  ...  52.567679   8.761280         1   
52713 2020-11-17 18:00:00      322  ...  58.408525   9.734754         0   
52714 2020-11-18 00:00:00      323  ...  58.700917   9.783486         0   
52715 2020-11-18 06:00:00      323  ...  64.222106  10.703684         0   
52716 2020-11-18 12:00:00      323  ...  70.329531  11.721589         0   

       34kt_radius  50kt_radius  64kt_radius  Rapid_NHC24  Outcome_Total  \
0           -999.0       -999.0       -999.0        False          False   
1           -999.0       -999.0       -999.0        False          False   
2           -999.0       -999.0       -999.0        False          False   
3           -999.0       -999.0       -999.0        False          False   
4           -999.0       -999.0       -999.0        False          False   
...            ...          ...          ...          ...            ...   
52712        122.5         37.5         12.5        False          False   
52713         90.0         15.0          0.0        False          False   
52714         67.5          0.0          0.0        False          False   
52715         35.0          0.0          0.0        False          False   
52716          0.0          0.0          0.0        False          False   

      Outcome_Switch Outcome_HoursToRapid  
0              False                  NaN  
1              False                  NaN  
2              False                  NaN  
3              False                  NaN  
4              False                  NaN  
...              ...                  ...  
52712          False                  NaN  
52713          False                  NaN  
52714          False                  NaN  
52715          False                  NaN  
52716          False                  NaN  

[52717 rows x 22 columns]

## Note 1: The format of the NHC HURDAT2.csv file

[From NHC] This dataset (known as Atlantic HURDAT2) is provided in comma-delimited, text format with six-hourly information on the location, maximum winds, central pressure, and (beginning in 2004) size of all known tropical cyclones and subtropical cyclones.

The format of the HURDAT file looks like this:

- **Storm Idenifier Row**
- *Strom Observation Row*
- *Strom Observation Row*
- *Strom Observation Row*
- **Storm Idenifier Row**
- *Strom Observation Row*
- *Strom Observation Row*
- *Strom Observation Row*
- *Strom Observation Row*

#### Important!
Within in an observation row, there is nothing that can identify which storm that the row belongs to. Thus, we need to add a storm identifier to each row for ease of use later. We do this in the `hurdat_lines_to_df` method below. 

See an example of the unchanged HURDAT data from Hurricane Irene below:

`AL092011,  IRENE,  39,  
20110821, 0000,  , TS, 15.0N,  59.0W,  45, 1006,  105, 0, 0,45, 0, 0, 0, 0, 0, 0, 0, 0,  
20110821, 0600,  , TS, 16.0N,  60.6W,  45, 1006,  130, 0, 0,80, 0, 0, 0, 0, 0, 0, 0, 0,  
20110821, 1200,  , TS, 16.8N,  62.2W,  45, 1005,  130, 0, 0,70, 0, 0, 0, 0, 0, 0, 0, 0,  
20110821, 1800,  , TS, 17.5N,  63.7W,  50,  999,  130,20, 0,70,30, 0, 0, 0, 0, 0, 0, 0,  
20110822, 0000,  , TS, 17.9N,  65.0W,  60,  993,  130,30,30,90,30, 0, 0,30, 0, 0, 0, 0,  
20110822, 0600,  , HU, 18.2N,  65.9W,  65,  990,  130,60,60,90,40,25,20,35,25, 0, 0, 0,  
20110822, 1200,  , HU, 18.9N,  67.0W,  70,  989,  160,60,60,90,40,25,20,35,25, 0, 0, 0,  
20110822, 1800,  , HU, 19.3N,  68.0W,  75,  988,  160,60,40,90,40,30,20,35,25, 0, 0, 0, `

The columns of the HURDAT file are as follows, refer to the link for more details.
- Date (YYYYMMDD)
- Time (24 hr)
- Record Identifier (see link for more info)
- Storm Status (Tropical Storm, Hurricane, etc. see link for more info)
- Latitude
- Longitude
- Maximum Sustained Wind Speed (kts)
- Minimum Pressure (mbar)
- 34 kt wind radii maximum extent in NE quadrant (in nautical miles)
- 34 kt wind radii maximum extent in SE quadrant (in nautical miles)
- 34 kt wind radii maximum extent in SW quadrant (in nautical miles)
- 34 kt wind radii maximum extent in NW quadrant (in nautical miles)
- 50 kt wind radii maximum extent in NE quadrant (in nautical miles)
- 50 kt wind radii maximum extent in SE quadrant (in nautical miles)
- 50 kt wind radii maximum extent in SW quadrant (in nautical miles)
- 50 kt wind radii maximum extent in NW quadrant (in nautical miles)
- 64 kt wind radii maximum extent in NE quadrant (in nautical miles)
- 64 kt wind radii maximum extent in SE quadrant (in nautical miles)
- 64 kt wind radii maximum extent in SW quadrant (in nautical miles)
- 64 kt wind radii maximum extent in NW quadrant (in nautical miles)

To learn more about the format of the HURDAT2 file, see the description at https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-nov2019.pdf